In [2]:
import requests
import json

### Задание 1
Посмотреть документацию к API GitHub, разобраться как вывести список репозиториев для конкретного пользователя,  
сохранить JSON-вывод в файле *.json.

Задаем токен авторизации и имя интересующего пользователя:

In [74]:
MY_TOKEN = '22773aed5285eb57f4cbe657c0b1bc0fe7036789'
user = 'KirillKras'

Создаем объект сессии с указанным токеном авторизации

In [49]:
session = requests.Session()
session.headers['Authorization'] = f'bearer {MY_TOKEN}'

Так как, согласно схеме валидации, запрос GraphQL API на получение репозиториев пользователя должен содержать информацию  
о количестве запрашиваемых репозиториев, сначала формируем запрос на получение количества репозитриев: 

In [101]:
query_count = '''
{
    user(login: "%s"){
        repositories {
            totalCount
        }
    }
}
''' % (user)

Выполняем запрос типа Query через объект сессии:

In [109]:
response = session.post('https://api.github.com/graphql', json={'query': query_count})

В полученном ответе, есои он был успешным, получаем количество репозиториев:

In [110]:
if response.ok:
    json_count = response.json()
    total_count = json_count['data']['user']['repositories']['totalCount']
print(f'У пользователя {user} количество репозиториев - {total_count}')

У пользователя KirillKras количество репозиториев - 11


Формируем запрос на получение репозиториев по имени пользователя и ранее установленному количеству,  
получаем имя репозитория, его краткое описание и реального владельца репозитория если форк:

In [105]:
query_repos = '''
{
    user(login: "%s"){
        repositories (first: %s) {
            edges {
                node {
                    name
                    description
                    parent {
                        owner {
                            login
                        }
                    }
                }
            }
        }
    }
}
''' % (user, total_count)

Выполняем запрос типа Query через объект сессии:

In [112]:
response = session.post('https://api.github.com/graphql', json={'query': query_repos})

Получаем ответ и выводим результаты в зависимости от наличия запрошенных полей: 

In [113]:
print(f'Список репозиториев пользователя {user}:')
json_repos = response.json()
for edge in json_repos['data']['user']['repositories']['edges']:
    result = f"Репозиторий: {edge['node']['name']} "
    if edge['node']['description']:
        result += f"\n\tкраткое описание: {edge['node']['description']}"
    if edge['node']['parent']:
        result += f"\n\tэто форк, реальный владелец {edge['node']['parent']['owner']['login']}"
    print(result)

Список репозиториев пользователя KirillKras:
Репозиторий: imagestore 
	краткое описание: Django gallery solution.
	это форк, реальный владелец hovel
Репозиторий: git-repo 
Репозиторий: alg_les1 
	краткое описание: Курс алгоритмы, лекция 1
Репозиторий: algs_structs 
	краткое описание: Курс "Алгоритмы и структуры данных"
Репозиторий: python_basic 
Репозиторий: Python_DS 
	краткое описание: Homework for "Python Data Science"
Репозиторий: in_math 
	краткое описание: курс "Введение в высшую математику"
Репозиторий: Linux 
	краткое описание: Курс "Linux. Рабочая станция"
Репозиторий: matan 
	краткое описание: Введение в математический анализ
Репозиторий: TerVer 
	краткое описание: Курс "Теория вероятности и мат. статистика"
Репозиторий: Parsing 


Сохраняем ответ в файл в формате json:

In [126]:
with open(f'{user}_repos.json', 'w') as f:
    f.write(json.dumps(['query_repos', json_repos], indent=4))